In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from scipy import stats
%matplotlib inline
from itertools import cycle

sns.set(style='ticks')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

In [2]:
df = pd.read_csv('teaching_method.csv')
df

,ID,Name,Sex,Math,Statistics,Psychology_test,Statistics_test1,Statistics_test2,Method
0,1,大村,m,dislike,like,13,6,10,C
1,2,本多,m,dislike,like,14,10,13,B
2,3,川崎,m,like,like,7,6,8,B
3,4,多村,m,like,like,12,10,15,A
4,5,松中,m,dislike,dislike,10,5,8,B
5,6,小久保,m,dislike,dislike,6,3,6,C
6,7,柴原,m,dislike,dislike,8,5,9,A
7,8,井手,m,dislike,dislike,15,9,10,D
8,9,田上,m,dislike,dislike,4,3,7,D
9,10,松田,m,like,dislike,14,3,3,D


## 独立な2群のt検定

In [6]:
statistics1_men = df[df['Sex']== 'm']['Statistics_test1'].values
statistics1_men

array([ 6, 10,  6, 10,  5,  3,  5,  9,  3,  3])

In [7]:
statistics1_women = df[df['Sex']== 'f']['Statistics_test1'].values
statistics1_women

array([11,  6, 11,  9,  7,  5,  8,  7,  7,  9])

帰無仮説: $\mu_1 = \mu_2$\
対立仮説: $\mu_1 \neq \mu_2$

検定統計量

$
t = \cfrac{\bar{X_1} - \bar{X_2}}{\sqrt{\cfrac{(n_1-1)\hat{\sigma}^{2}_{1} + (n_2-1)\hat{\sigma}^{2}_{2}}{n_1+n_2-2} \left( \cfrac{1}{n_1} + \cfrac{1}{n_2}\right) }}
$

自由度$df=n_1+n_2-1$のt分布に従う。

In [10]:
print(np.mean(statistics1_men))
print(np.mean(statistics1_women))

print(np.var(statistics1_men, ddof=1))
print(np.var(statistics1_women, ddof=1))

6.0
8.0
7.777777777777778
4.0


$
\hat{\sigma}_{pooled} = \sqrt{\cfrac{(n_1-1)\hat{\sigma}^{2}_{1} + (n_2-1)\hat{\sigma}^{2}_{2}}{n_1+n_2-2} }
$

In [16]:
pool_std = np.sqrt( ( (len(statistics1_men)-1)*(np.var(statistics1_men, ddof=1)) \
                   + (len(statistics1_women)-1)*(np.var(statistics1_women, ddof=1)) )\
                   / (len(statistics1_men)+len(statistics1_women)-2) ) 
pool_std

2.4267032964268394

In [17]:
t_lower = pool_std * np.sqrt(1/len(statistics1_men) + 1/len(statistics1_women))
t_lower

1.0852547064066471

In [19]:
t_upper = np.mean(statistics1_men) - np.mean(statistics1_women)
t_upper

-2.0

In [20]:
t = t_upper/t_lower
t

-1.8428853505018534

In [22]:
# 自由度18のt分布で下側確率0.025となるtの値
print(stats.t.ppf(0.025, 18))
# 自由度18のt分布で上側確率0.025となるtの値
print(stats.t.isf(0.025, 18))

-2.10092204024096
2.10092204024096


よって検定統計量は棄却域に入らないので、帰無仮説を棄却できない。\
つまり男女間で平均値に有意な差ないということ。\
この標本数では、平均値の差が2点であるということは十分にありえるということ。

In [25]:
#独立な2群のt検定
t, p = stats.ttest_ind(statistics1_men, statistics1_women)
stats.ttest_ind(statistics1_men, statistics1_women)

Ttest_indResult(statistic=-1.8428853505018534, pvalue=0.08187801462410679)

## 分散の等質性の検定

In [26]:
classa = np.array([54,55,52,48,50,38,41,40,53,52])
classb = np.array([67,63,50,60,61,69,43,58,36,29])

stats.bartlett(classa, classb)

BartlettResult(statistic=4.589710897779921, pvalue=0.032164439095344693)

帰無仮説が棄却される。-> 分散の等質性の仮定は満たされないということ。\
つまり、独立な2群のt検定を行えない。

## Welchの検定
分散が等質でないときに使用する

In [27]:
stats.ttest_ind(classa, classb, equal_var=False)

Ttest_indResult(statistic=-1.1191356939670116, pvalue=0.2837877174603801)

５％で有意にならない。つまり、平均値に有意な差があるとはいえない。

## 対応のあるt検定

検定統計量

$
t = \cfrac{\bar{D}} {\hat{\sigma}_D / \sqrt{n}}
$

自由度$df=n-1$のt分布に従う。

In [29]:
change = df['Statistics_test2'].values - df['Statistics_test1'].values
change

array([ 4,  3,  2,  5,  3,  3,  4,  1,  4,  0,  7,  8,  7,  2,  5,  0, -1,
        5,  0, -2])

In [30]:
t_lower = np.std(change, ddof=1) / np.sqrt(len(change))
t_lower

0.6198471798078857

In [32]:
t_upper = np.mean(change)
t_upper

3.0

In [33]:
t = t_upper / t_lower
t

4.8399026368560945

In [34]:
# 自由度9のt分布で下側確率0.025となるtの値
print(stats.t.ppf(0.025, 9))
# 自由度9のt分布で上側確率0.025となるtの値
print(stats.t.isf(0.025, 9))

-2.262157162740992
2.262157162740992


帰無仮説は棄却される。\
指導の前後でテスト1とテスト2の得点の平均値に5%水準で有意差が見られた。

In [35]:
# 対応のあるt検定
stats.ttest_rel(df['Statistics_test1'].values, df['Statistics_test2'].values)

Ttest_relResult(statistic=-4.8399026368560945, pvalue=0.00011378795008861286)

In [36]:
# 対応なしとみなした場合
# まず、分散の等質性を調べる
stats.bartlett(df['Statistics_test1'].values, df['Statistics_test2'].values)

BartlettResult(statistic=3.9332088763242945, pvalue=0.0473415291779616)

In [37]:
# p値が5%以下(等質性がない)なのでWelchの検定をする
stats.ttest_ind(df['Statistics_test1'].values, df['Statistics_test2'].values, equal_var=False)

Ttest_indResult(statistic=-2.7629564910487767, pvalue=0.009429462533670687)

こちらも有意差ありという結果になった。\
しかし、対応ありの場合のほうがp値が小さい。

# 練習問題
#### 1

In [45]:
st_like = df[df['Statistics']=='like']['Statistics_test1'].values
print(st_like)
st_dislike = df[df['Statistics']=='dislike']['Statistics_test1'].values
print(st_dislike)

[ 6 10  6 10 11  6 11  7]
[5 3 5 9 3 3 9 5 8 7 7 9]


In [47]:
# 等質性の検定
stats.bartlett(st_like, st_dislike)

BartlettResult(statistic=0.006200381166804912, pvalue=0.937237441000522)

In [48]:
# 等質性がないのでWelchの検定をする
stats.ttest_ind(st_like, st_dislike, equal_var=False)

Ttest_indResult(statistic=2.1342261113376138, pvalue=0.04922015258617567)

有意差あり。つまり、好き嫌いでテストの点が変わる

#### 2

In [49]:
psy_men = df[df['Sex']=='m']['Psychology_test'].values
print(psy_men)
psy_women = df[df['Sex']=='f']['Psychology_test'].values
print(psy_women)

[13 14  7 12 10  6  8 15  4 14]
[ 9  6 10 12  5 12  8  8 12 15]


In [51]:
# 等質性の検定
stats.bartlett(psy_men, psy_women)

BartlettResult(statistic=0.4150856573275788, pvalue=0.5193994478045708)

In [52]:
# 等質性がないのでWelchの検定をする
stats.ttest_ind(psy_men, psy_women, equal_var=False)

Ttest_indResult(statistic=0.38358697133460806, pvalue=0.7059891893345245)

有意差はない。つまり男女間での心理学テストの結果の平均値は誤差。

#### 3

In [53]:
before = np.array([61,50,41,55,51,48,46,55,65,70])
after = np.array([59,48,33,54,47,52,38,50,64,63])

In [54]:
stats.ttest_rel(before, after)

Ttest_relResult(statistic=2.8465424181483328, pvalue=0.019197155453924926)

有意差あり。つまりプログラムの前と後での体重に変化があったということ。